In [1]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20210119~18.04.1).
dirmngr is already the newest version (2.2.4-1ubuntu1.4).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,696 B of archives.
After this operation, 153 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.12ubuntu0.2 [1,696 B]
Fetched 1,696 B in 0s (5,978 B/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend:

In [2]:
!pip install clickhouse-driver

     |████████████████████████████████| 481kB 8.0MB/s 


In [60]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('db_sales',), ('default',), ('system',)]

Загрузка датасета

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

In [14]:
dateparser = lambda x: datetime.strptime(x, "%d.%m.%Y")
df = pd.read_csv('data.csv',sep=';',decimal=',',parse_dates = ['date'], date_parser=dateparser)

In [15]:
print(df.head(5))
print(df.info())

        date  region product  amount  delta
0 2020-01-01       1       a      14   15.0
1 2020-01-02       2       b      50   19.7
2 2020-01-03       3       c      97   12.9
3 2020-01-04       1       d      29   16.8
4 2020-01-05       2       a      68   19.4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     20 non-null     datetime64[ns]
 1   region   20 non-null     int64         
 2   product  20 non-null     object        
 3   amount   20 non-null     int64         
 4   delta    20 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 928.0+ bytes
None


Добавление новой БД и запись значений

In [16]:
client.execute('CREATE DATABASE db_sales')

[]

In [17]:
client.execute('SHOW DATABASES')

[('db_sales',), ('default',), ('system',)]

In [62]:
client = Client(host='localhost', user='default', port='9000', database='db_sales',settings={'use_numpy': True})

In [22]:
client.execute('DROP TABLE IF EXISTS sales')

[]

In [23]:
client.execute('CREATE TABLE sales (date Date, region Int32, product String, amount Int32, delta Float32  ) ENGINE = Memory')

[]

In [24]:
client.execute('SHOW TABLES FROM db_sales')

[('sales',)]

In [25]:
client.execute("INSERT INTO sales VALUES", df.to_dict('records'))

20

In [26]:
client.execute('SELECT * FROM sales LIMIT 5')

[(datetime.date(2020, 1, 1), 1, 'a', 14, 15.0),
 (datetime.date(2020, 1, 2), 2, 'b', 50, 19.700000762939453),
 (datetime.date(2020, 1, 3), 3, 'c', 97, 12.899999618530273),
 (datetime.date(2020, 1, 4), 1, 'd', 29, 16.799999237060547),
 (datetime.date(2020, 1, 5), 2, 'a', 68, 19.399999618530273)]

In [73]:
def select_clickhouse(sql):
  return client.query_dataframe(sql)

In [74]:
sql = '''SELECT * FROM sales LIMIT 5'''

In [75]:
select_clickhouse(sql)

,date,region,product,amount,delta
0,2020-01-01,1,a,14,15.000000
1,2020-01-02,2,b,50,19.700001
2,2020-01-03,3,c,97,12.900000
3,2020-01-04,1,d,29,16.799999
4,2020-01-05,2,a,68,19.400000


## Преобразование типов

In [88]:
sql = '''SELECT s.*, toInt32(round(s.delta,0)) as delta_int FROM sales as s LIMIT 5'''

In [89]:
select_clickhouse(sql)

,date,region,product,amount,delta,delta_int
0,2020-01-01,1,a,14,15.000000,15
1,2020-01-02,2,b,50,19.700001,20
2,2020-01-03,3,c,97,12.900000,13
3,2020-01-04,1,d,29,16.799999,17
4,2020-01-05,2,a,68,19.400000,19


## Условные функции

In [94]:
sql ='''SELECT multiIf(s.product = 'a','aaa',s.product = 'b','bbb','ccc') as product_name FROM sales as s LIMIT 5'''

In [95]:
select_clickhouse(sql)

,product_name
0,aaa
1,bbb
2,ccc
3,ccc
4,aaa


## Агрегатные функции

In [96]:
sql ='''SELECT corr(s.amount,s.delta) as corr FROM sales as s'''

In [97]:
select_clickhouse(sql)

,corr
0,-0.373312


In [101]:
sql ='''SELECT topK(2)(s.product) FROM sales as s'''

In [102]:
select_clickhouse(sql)

,topK_2__product_
0,"[a, b]"


In [105]:
sql ='''SELECT argMin(s.product, amount) as argMin, argMin(tuple(s.product, s.amount)) as argMin_tuple FROM sales as s'''

In [106]:
select_clickhouse(sql)

,argMin,argMin_tuple
0,a,"(a, 14)"


In [121]:
sql ='''SELECT simpleLinearRegression(s.amount,s.delta) as linearRegression FROM sales as s'''

In [122]:
select_clickhouse(sql)

,linearRegression
0,"(-0.03912577754180567, 18.034076274781373)"


In [144]:
sql = '''WITH test as (SELECT s.amount, s.product, if(s.product ='a',1,0) as product_code FROM sales as s WHERE product IN ('a','b'))
SELECT studentTTest(t.amount, t.product_code) as t_test  FROM  test as t'''

In [145]:
select_clickhouse(sql)

,t_test
0,"(0.38800371725178673, 0.7081336841547918)"
